In [1]:
import json
import numpy as np
import gc

In [2]:
gc.disable()
gc.isenabled()

False

In [3]:
# Import Data

train_data,test_data = [],[]
with open('train_data.json',encoding='utf-8') as read_file:
    for line in read_file:
        train_data.append(json.loads(line))
    read_file.close()

with open('test_data.json',encoding='utf-8') as read_file:
    for line in read_file:
        test_data.append(json.loads(line))
    read_file.close()

In [4]:
train_review = train_data[0]['Reviews']
train_rating = np.asarray(train_data[0]['Ratings']).astype('float16')

test_review = test_data[0]['Reviews']
test_rating = np.asarray(test_data[0]['Ratings']).astype('float16')

In [5]:
train_rating.shape

(25000,)

In [6]:
for i in range(train_rating.shape[0]):
    if train_rating[i]>5:
        train_rating[i]=1
    else:
        train_rating[i]=0
        
np.max(train_rating)

1.0

### Clean Data

In [7]:
import Data_Preprocss as clean

In [8]:
train = clean.get_preocessed_data(train_review)
test = clean.get_preocessed_data(test_review)

In [9]:
print(len(train[0]))

499


### Vectorize data

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_features=12000)

In [12]:
data = ((cv.fit_transform(train).toarray()).astype('float16'))

In [13]:
data.shape

(25000, 12000)

### Build model

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *

In [20]:
model = Sequential()

In [21]:
model.add(Dense(64,activation='relu',input_shape=(12000,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [22]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                768064    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 770,689
Trainable params: 770,689
Non-trainable params: 0
_________________________________________________________________


In [24]:
hist=model.fit(data,train_rating,batch_size=512,epochs=3,validation_split=0.1)

Epoch 1/3
44/44 [==============================] - 4s 86ms/step - loss: 0.4418 - accuracy: 0.8147 - val_loss: 0.2418 - val_accuracy: 0.9416
Epoch 2/3
44/44 [==============================] - 5s 107ms/step - loss: 0.2542 - accuracy: 0.9105 - val_loss: 0.5284 - val_accuracy: 0.8036
Epoch 3/3
44/44 [==============================] - 3s 78ms/step - loss: 0.1950 - accuracy: 0.9309 - val_loss: 0.7901 - val_accuracy: 0.7392
